# Self Attention (ViT 2020)

[Visual Transformers: Token-based Image Representation and Processing for Computer Vision (Wu et al., 2020)](https://arxiv.org/abs/2006.03677)

[Реализация](https://github.com/lucidrains/vit-pytorch)

[Блог-пост разбор  ViT](https://towardsdatascience.com/implementing-visualttransformer-in-pytorch-184f9f16f632)


**Vision Transformer** — это модель для классификации изображений, которая использует архитектуру трансформера. Попробуем разобраться, как она работает.

В 2020 году стали появляться работы, где модели на базе архитектур трансформер смогли показать результаты лучше, чем у **CNN** моделей.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cited_vit_accuracy.png"  width="650"></center>


<center><em>Source: <a href="https://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020</a></em></center>

BiT — это baseline модель на базе **ResNet**, ViT — **Visual Transformer**



### Недостатки сверточного слоя

Авторы практически полностью отказались от использования сверток,  заменив их слоями **self-attention**.  Попробуем понять, почему это сработало.

Добавляя в модель свёрточный слой, мы руководствуемся резонным предположением: чем ближе пиксели на изображении, тем больше будет их взаимное влияние.

В большинстве случаев это работает:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cnn_ok.png" width="700"></center>

 - На слое n (красный) активируются нейроны, которые реагируют на морду и на хвост кота.

 - В карте активаций их выходы оказываются рядом, и в слое n + 1 (синий) они попадают в одну свертку, которая активируется на объектах типа "кот".

Так случается часто, но не всегда:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cnn_fail.jpg"  width="700"></center>

На этом изображении активации нейронов, реагирующих на морду и хвост, не попадут в одну свертку на следующем слое. Это может привести к тому, что нейрон, обучившийся реагировать на кошек, не активируется.

Причиной этого является допущение ([Inductive bias](https://en.wikipedia.org/wiki/Inductive_bias)) о взаимном влиянии соседних пикселей.

### Self-attention

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/global_attention.png"  width="900"></center>

**Self-attention** слой лишен этого недостатка. Он обучается оценивать взаимное влияние входов друг на друга. Но как применить его к изображениям?

В статье [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020)](https://arxiv.org/pdf/2010.11929.pdf) предлагается разбивать картинки на кусочки (patches) размером 16x16 пикселей и подавать их на вход модели.

Проделаем это:

In [ ]:
URL = "https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cat.jpeg"
!wget -q $URL -O image.jpg

Преобразуем изображение в тензор, порежем на фрагменты и отобразим их, используя image_grid

In [ ]:
from torchvision import utils, transforms
import matplotlib.pyplot as plt
import torch
from PIL import Image

img = Image.open("image.jpg")

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

img = transform(img)
patches = []
sz = 64
for r in range(0, img.shape[1], sz):
    for c in range(0, img.shape[2], sz):
        patches.append(img[:, r : r + sz, c : c + sz])

patches = torch.stack(patches).type(torch.float)

img_grid = utils.make_grid(patches, pad_value=10, normalize=True, nrow=4)
plt.imshow(transforms.ToPILImage()(img_grid).convert("RGB"))
plt.axis("off")
plt.show()

На вход модели они поступят в виде вектора:

In [ ]:
plt.figure(figsize=(18, 6))
img_grid = utils.make_grid(patches, pad_value=10, normalize=True, nrow=256 // 16)
plt.imshow(transforms.ToPILImage()(img_grid).convert("RGB"))
plt.axis("off");

Затем последовательность из фрагментов изображения передается в модель, где после ряда преобразований попадает на вход слоя **self-attention**:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/self_attention.png"  width="900"></center>

Картинки приведены исключительно для наглядности, в действительности слой работает с векторами признаков, которые не визуализируются столь очевидно. Однако кэффициенты, с которыми складываются вектора-признаков, отражают важность каждого с учетом всех остальных входов.



## Сравнение со сверткой

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/conv_vs_self_attention1.png" width="400">

При свертке каждый признак умножается на свой вес, и затем они суммируются. Важно что вклад взвешенных признаков в сумму не зависит от контекста.

То есть ягода клубники, лежащая на столе (где рядом с ней может быть все, что угодно), даст такой же вклад в сумму, как и ягода с клубничного куста.



<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/conv_vs_self_attention2.png"  width="900"></center>

Слой self-attention выполняет ту же задачу, что и свертка: получает на вход вектор признаков и возвращает другой, более информативный.  Но делает это более умно:





<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/conv_vs_self_attention3.png"  width="900"></center>

*Вместо чисел здесь вектора, но принципильно это ничего не меняет, можно применить self-attention и к отдельным признакам (яркостям, пикселям), просто для это потребуется очень много ресурсов.*

**Каждый признак** участвует в каждой сумме, а не только те, что попали в рецептивное поле фильтра.
Кроме этого, суммируются они с коэффициентами $a$, которые **зависят от входов** и различны для каждой суммы.

Для получения этих коэффициентов и нужна большая часть слоя self-attention. На рисунке выделено красным.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/conv_vs_self_attention5.png"  width="900"></center>

#### Как получить веса внимания?

In [ ]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim**0.5)
        attention = scores.softmax(dim=2)
        print("Scores shape", scores.shape)
        weighted = torch.bmm(attention, values)
        return weighted

In [ ]:
embed_dim = 256
self_attention_layer = SelfAttention(embed_dim)
dummy_x = torch.randn(1, 4 * 4, embed_dim)  # Batch_size x Sequence_len x Embedding_size
out = self_attention_layer(dummy_x)
print(out.shape)

#### Соображения относительно размера patch

Трансформеры работают с последовательностями за счёт механизма внимания (**self-attention**). И чтобы подать на вход изображение, требуется превратить его в последовательность.

Сделать это можно разными способами, например, составить последовательность из всех пикселей изображения. Её длина $n =  H*W$ (высота на ширину)

[Сложность вычисления](https://www.researchgate.net/figure/Compare-the-computational-complexity-for-self-attention-where-n-is-the-length-of-input_tbl7_347999026) одноголового слоя **self-attention** $O(n^2 d )$,  где $n$ — число токенов и $d$ — размерность входа (embedding)  (для любознательных расчеты [тут](https://stackoverflow.com/questions/65703260/computational-complexity-of-self-attention-in-the-transformer-model)).

То есть для квадратных изображений $(H==W)$ получим $O(H^3 d )$

1. Такой подход будет очень вычислительно сложен.

2. Интуитивно понятно, что кодировать каждый пиксель относительно большим embedding-ом не очень осмысленно.


*Для тех, кто забыл, напомним что $O()$ — это Big O notation, которая отражает ресурсы, требуемые для вычисления. Так для $O(1)$ — время вычисления будет постоянным, вне зависимости от количества данных, а для $O(N)$ — расти пропорционально количеству данных.*


Разберём на примере: Допустим, мы используем трансформер для предложения длиной в 4 слова — "Мама мылом мыла раму" => у нас есть `4 токена`. Закодируем их в *embeddings* с размерностью `256`. Потребуется порядка $4^2*256 = 4096$ операций.

А теперь попробуем провернуть то же самое для картинки размерами 256 на 256.
Количество токенов

 $256^3*256  = 256^4 =  4 294 967 296 $. Упс... Кажется, нам так никаких ресурсов не хватит — трансформеры с картинками использовать.



Посчитаем сложность для картинки размером 256x256, разбитой на кусочки по 16px. при том же размере токена (256) $n = 16$.
$16^2*256 = 256^2 = 65536 $. И впрямь! ~65000 раз меньше ресурсов требуется.

[Как устроен  self-attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

[Self-attention слой в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)

### Position embedding

Не теряем ли мы важной информации, разбивая изображение на фрагменты? На первый взгляд кажется, что модель сможет научиться восстанавливать порядок, в котором фрагменты шли в исходном изображении.

Всегда ли?

Рассмотрим пример изображения, где нет ярко выраженной текстуры:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/positional_transformer_explanation.png"  width="500"></center>

На рисунке а) наковальня падает на ребенка, на рисунке б) ребенок прыгает на наковальне.

Суть принципиально отличается, но что будет, если составить из фрагментов любого изображения набор патчей:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/positional_vec_transformer_explanation.png"  width="500"></center>

Восстановить по нему можно будет любой из вариантов!

Так как **self-attention** блок никак не кодирует позицию элемента на входе, то важная информация потеряется.

Чтобы избежать таких потерь, информацию, кодирующую позицию фрагмента (patch),  добавляют к входным данным **self-attention** слоя в явном виде.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L08/out/linear_projection_of_flattened_patches.png"  width="600"></center>

[Методы для кодирования позиции](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)

## Архитектура ViT

Теперь мы можем грузить наши изображения в **Vi**sual **T**ransformer.

**Self-attention** блок мы разобрали, остальные блоки модели нам знакомы:

> **MLP** (Multi layer perceptron) — Блок из одного или нескольких линейных слоев

> **Norm** — Layer Normalization

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L08/out/visual_transformer_architecture.png"  width="1000"></center>
<center><em>Архитектура Visual Transformer </em></center>



1.   Изображение режется на фрагменты (patch).
2.   Фрагменты (patch) подвергаются линейной проекции с помощью **MLP**.
3.   С полученными на выходе **MLP** векторами конкатенируются **positional embeddings** (кодирующие информацию о позиции path, как и в обычном трансформере для текста).
4. К полученным векторам добавляют еще один **0***, который называют **class embedding**.

Любопытно, что для предсказания класса используется только выход. Он соответствует дополнительному **class embedding**.  Остальные выходы (а для каждего токена в трансформере есть свой выход) отбрасываются за ненадобностью.

В финале этот специальный токен **0*** прогоняют через **MLP** и предсказывают классы.

Попробуем провести аналогию со свертками. Если рассмотреть сверточную сеть без слоев пулинга и других способов изменения пространственных размеров карт признаков, то можно считать такую сеть механизмом постепенно улучшаюшем качество признаков.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.0/L08/out/selfattention_feature_flow.png"  width="900"></center>

## Предсказание с помощью ViT


Используем пакет [ViT PyTorch](https://pypi.org/project/pytorch-pretrained-vit/)



In [ ]:
!pip install -q pytorch_pretrained_vit

В пакете доступны несколько [предобученных моделей](https://github.com/lukemelas/PyTorch-Pretrained-ViT#loading-pretrained-models):

B_16, B_32, B_16_imagenet1k, ...



In [ ]:
from pytorch_pretrained_vit import ViT
from torchvision import transforms

model = ViT("B_16_imagenet1k", pretrained=True)
model.eval()

In [ ]:
# Load image
!wget  https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/capybara.jpg

In [ ]:
capybara_in_pil = Image.open("capybara.jpg")
transforms = transforms.Compose(
    [
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)
capybara_in_tensor = transforms(capybara_in_pil)
print(capybara_in_tensor.shape)  # torch.Size([1, 3, 384, 384])

# Classify
with torch.no_grad():
    outputs = model(capybara_in_tensor.unsqueeze(0))
print(outputs.shape)  # (1, 1000)

Давайте посмотрим, что нам предсказывает ViT. Для этого подгрузим dict с переводом индексов в человеческие названия:

И, собственно, переведем индекс в название:

In [ ]:
top3 = outputs[0].topk(3).indices
top3 = top3.tolist()


print("Top 3 predictions:")
for class_num in top3:
    print(class_num, classes[class_num])
display(capybara_in_pil.resize((384, 384)))

Ну что ж, почти (капибар в классах ImageNet 1k, как вы могли догадаться, просто нет).

## Обучение ViT

### Объем данных и ресурсов

Как следует из текста [статьи](https://arxiv.org/abs/2010.11929), **ViT**, обученный на **ImageNet**, уступал baseline CNN-модели
на базе сверточной сети (**ResNet**). И только при увеличении датасетов больше, чем **ImageNet**, преимущество стало заметным.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cited_vit_accuracy.png"  width="400"></center>

<center><em>Source: <a href="https://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020)</a></em></center>


Вряд ли в вашем распоряжении окажется датасет, сравнимый с [JFT-300M](https://paperswithcode.com/dataset/jft-300m) (300 миллионов изображений),
и GPU/TPU ресурсы, необходимые для обучения с нуля (*it could be trained using a standard cloud TPUv3 with 8 cores in approximately 30 days*)

Поэтому для работы с пользовательскими данными используется техника дообучения ранее обученной модели на пользовательских данных (**fine-tuning**).

## DeiT: Data-efficient Image Transformers

Для практических задач рекомендуем использовать эту реализацию. Авторы предлагают подход, благодаря которому становится возможным обучить модель на стандартном **ImageNet** (ImageNet1k) на одной рабочей станции за 3 дня.

*We train them on a single computer in less than 3 days. Our reference vision transformer (86M parameters) achieves top-1 accuracy of 83.1% (single-crop evaluation) on ImageNet with no external data.*

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/cited_deit_vit.png"  width="700"></center>

<center><em>Source: <a href="https://arxiv.org/abs/2012.12877">Training data-efficient image transformers & distillation through attention</a></em></center>



Разбор этого материала уже не входит в наш курс и рекомендуется к самостоятельному изучению.

Дополнительно:
[Distilling Transformers: (DeiT) Data-efficient Image Transformers](https://towardsdatascience.com/distilling-transformers-deit-data-efficient-image-transformers-61f6cd276a03)

Статьи, предшествовавшие появлению **ViT**:

[Non-local Neural Networks](https://arxiv.org/abs/1711.07971)

[CCNet: Criss-Cross Attention for Semantic Segmentation](https://arxiv.org/abs/1811.11721)






### Использование ViT с собственным датасетом

Для использования **ViT** с собственными данными рекомендуем не обучать собственную модель с нуля, а использовать уже предобученную.

Рассмотрим этот процесс на примере. Есть предобученный на **ImageNet** **Visual Transformer**, например: [deit_tiny_patch16_224](https://github.com/facebookresearch/deit)

И мы хотим использовать ее со своим датасетом, который может сильно отличаться от **ImageNet**.

Для примера возьмем **CIFAR-10**.



Загрузим модель. Как указано на [github](https://github.com/facebookresearch/deit), модель зависит от библиотеки [timm](https://fastai.github.io/timmdocs/), которую нужно установить.

In [ ]:
!pip install -q timm

Теперь загружаем модель с [pytorch-hub](https://pytorch.org/hub/):

In [ ]:
import torch

model = torch.hub.load(
    "facebookresearch/deit:main", "deit_tiny_patch16_224", pretrained=True
)

Убедимся, что модель запускается.
Загрузим изображение:

In [ ]:
!wget  https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/capybara.jpg

И подадим его на вход трансформеру:

In [ ]:
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import torchvision.transforms as T
from PIL import Image

pil = Image.open("capybara.jpg")

# create the data transform that DeiT expects
imagenet_transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    ]
)

out = model(imagenet_transform(pil).unsqueeze(0))
print(out.shape)
pil.resize((224, 224))

Чтобы использовать модель с **CIFAR-10**, нужно поменять количество выходов слоя, отвечающих за классификацию. Так как в **CIFAR-10** десять классов, а в **ImageNet** — тысяча.

Чтобы понять, как получить доступ к последнему слою, выведем структуру модели:


In [ ]:
print(model)

Видим, что последний слой называется head и, судя по количеству параметров на выходе (1000), которое совпадает с количеством классов **ImageNet**, именно он отвечает за классификацию.

In [ ]:
print(model.head)

Заменим его слоем с 10-ю выходами по количеству классов в CIFAR-10.

In [ ]:
model.head = torch.nn.Linear(192, 10, bias=True)

Убедимся, что модель не сломалась.

In [ ]:
out = model(imagenet_transform(pil).unsqueeze(0))
print(out.shape)

Теперь загрузим **CIFAR-10** и проверим, как дообучится модель

In [ ]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

cifar10 = CIFAR10(root="./", train=True, download=True, transform=imagenet_transform)

# We use only part of CIFAR10 to reduce training time
trainset, _ = torch.utils.data.random_split(cifar10, [10000, 40000])
train_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = CIFAR10(root="./", train=False, download=True, transform=imagenet_transform)
test_loader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

 Проведем стандартный цикл обучения.

In [ ]:
from torch import nn
from tqdm.notebook import tqdm_notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(model, train_loader, optimizer, num_epochs=1):
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch in tqdm_notebook(train_loader):
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

Дообучаем (**fine tune**) только последний слой модели, который мы изменили.

In [ ]:
import torch.optim as optim

model.to(device)
optimizer = optim.SGD(model.head.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, optimizer)

Проверим точность, на всей тестовой подвыборке **CIFAR-10**.

In [ ]:
@torch.inference_mode()
def accuracy(model, testloader):
    correct = 0
    total = 0
    for batch in testloader:
        images, labels = batch
        outputs = model(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
    return correct / total

In [ ]:
print(f"Accuracy of fine-tuned network : {accuracy(model, test_loader):.2f} ")

Дообучив последний слой на одной эпохе с использованием 20% данных, мы получили точность ~0.75

Если дообучить все слои на 2-х эпохах, можно получить точность порядка 0.95.

Это результат намного лучше чем тот, что мы получали на семинарах.

Для этого потребуется порядка 10 мин (на GPU). Сейчас мы этого делать не будем.


И одной из причин того, что обучение идет относительно медленно, является увеличение изображений размером 32x32 до 224x224.

Если бы мы использовали изображения **CIFAR-10** в их родном размере, мы бы не потеряли никакой информации, но могли бы в разы ускорить обучение.


### Изменение размеров входа ViT

На первый взгляд, ничего не мешает это сделать: **self-attention** слой работает с произвольным количеством входов.

Давайте посмотрим, что будет, если подать на вход модели изображение, отличное по размерам от 224x224.

Для этого перезагрузим модель:

In [ ]:
def get_model():
    model = torch.hub.load(
        "facebookresearch/deit:main", "deit_tiny_patch16_224", pretrained=True
    )
    model.head = torch.nn.Linear(192, 10, bias=True)
    return model


model = get_model()

И уберем из трансформаций Resize:

In [ ]:
cifar_transform = T.Compose(
    [
        # T.Resize((224, 224)),    don't remove this line
        T.ToTensor(),
        T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    ]
)

# Change transformation in base dataset
cifar10.transform = cifar_transform
first_img = trainset[0][0]

model.to(torch.device("cpu"))
try:
    out = model(first_img.unsqueeze(0))
except Exception as e:
    print("Exception:", e)

Получаем ошибку.

Ошибка возникает в объекте [PatchEmbed](https://huggingface.co/spaces/Andy1621/uniformer_image_demo/blob/main/uniformer.py#L169), который превращает изображение в набор эмбеддингов.

У объекта есть свойство `img_size`, попробуем просто поменять его:

In [ ]:
model.patch_embed.img_size = (32, 32)
try:
    out = model(first_img.unsqueeze(0))
except Exception as e:
    print("Exception:", e)

Получаем новую ошибку.

И возникает она в строке
`x = self.pos_drop(x + self.pos_embed)`

x — это наши новые эмбеддинги для CIFAR-10 картинок

Откуда взялось число 5?

4 — это закодированные фрагменты (patch) для картинки 32х32, их всего 4 (16x16) + один embedding для предсказываемого класса(class embedding).

А 197 — это positional encoding — эмбеддинги, кодирующие позицию элемента. Они остались от **ImageNet**.

Так как в ImageNet картинки размера 224x224, то в каждой помещалось 14x14 = 196 фрагментов и еще embedding для класса, итого 197 позиций.



Эмбеддинги для позиций доступны через свойство:

In [ ]:
model.pos_embed.data.shape

Теперь нам надо изменить количество pos embeddings так, чтобы оно было равно 5  (количество patch + 1).
Возьмем 5 первых:

In [ ]:
model.pos_embed.data = model.pos_embed.data[:, :5, :]
out = model(first_img.unsqueeze(0))
print(out.shape)

Заработало!

Теперь обучим модель. Так как изображения стали намного меньше, то мы можем увеличить размер batch и использовать весь датасет. Также будем обучать все слои, а не только последний.

In [ ]:
cifar10.transform = cifar_transform
train_loader = DataLoader(cifar10, batch_size=512, shuffle=True, num_workers=2)

# Now we train all parameters because model altered
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, optimizer)

Сильно быстрее.
Посмотрим на результат:

In [ ]:
testset.transform = cifar_transform
print(f"Accuracy of altered network : {accuracy(model,test_loader):.2f} ")

Сильно хуже.

Это можно объяснить тем, что  маленькие patch  ImageNet(1/196) семантически сильно отличаются от четвертинок картинок из CIFAR-10 (1/4).

Но есть и другая причина: мы взяли лишь первые 4 pos_embedding а остальные отбросили. В итоге модель вынуждена практически заново обучаться работать с малым pos_embedding, и двух эпох для этого мало.

Зато теперь мы можем использовать модель с изображениями любого размера.